Righnow candidate is : SVR with PCA n_components = 10 with results 0.23706564506631833

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, TheilSenRegressor,RANSACRegressor,HuberRegressor
from sklearn.metrics import r2_score

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
test.head(5)

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,1,az,v,n,f,d,t,a,w,0,...,0,0,0,1,0,0,0,0,0,0
1,2,t,b,ai,a,d,b,g,y,0,...,0,0,1,0,0,0,0,0,0,0
2,3,az,v,as,f,d,a,j,j,0,...,0,0,0,1,0,0,0,0,0,0
3,4,az,l,n,f,d,z,l,n,0,...,0,0,0,1,0,0,0,0,0,0
4,5,w,s,as,c,d,y,i,m,0,...,1,0,0,0,0,0,0,0,0,0


In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectFromModel

In [5]:
train_labels = train.iloc[:,1]
train_feats = train.iloc[:, 2:]

test_labels = test.iloc[:,1]
test_feats = test.iloc[:, 1:]

In [20]:
train_feats_encode = []
train_feats_encode.append(train.iloc[:, 0])
for i in range(train_feats.shape[1]):
    arr = train_feats.iloc[:, i]
    if arr.dtype == 'O':        
        lblencod = LabelEncoder()
        arr = lblencod.fit_transform(arr)
    train_feats_encode.append(arr)
train_feats_encode = np.array(train_feats_encode).T

test_feats_encode = []
test_feats_encode.append(test.iloc[:, 0])
for i in range(test_feats.shape[1]):
    arr = test_feats.iloc[:, i]
    if arr.dtype == 'O':        
        lblencod = LabelEncoder()
        arr = lblencod.fit_transform(arr)
    test_feats_encode.append(arr)
test_feats_encode = np.array(test_feats_encode).T

In [25]:
cellen=['ID', 'X47','X95','X314','X315','X232','X119','X311','X76','X329','X238','X340','X362','X137']
train_feats_encode = train[cellen].as_matrix()
test_feats_encode = test[cellen].as_matrix()

Start to do the ensemble method

***XGBOOST***

In [26]:
from xgboost import XGBRegressor

***SVR***

In [27]:
from sklearn.svm import SVR

***Linear Regression***

In [28]:
from sklearn.linear_model import LinearRegression

*** SGD Regressor ***

In [29]:
from sklearn.linear_model import SGDRegressor

*** Keras Deep Learning ***

In [30]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping, ModelCheckpoint
# define custom R2 metrics for Keras backend
from keras import backend as K
# to tune the NN
from keras.constraints import maxnorm
from keras.optimizers import SGD, Adam
act_func = 'tanh'
input_dims = train_feats_encode.shape[1]

def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true - y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [31]:
# base model architecture definition
def model():
    model = Sequential()
    #input layer
    model.add(Dense(input_dims, input_dim=input_dims))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    # hidden layers
    model.add(Dense(input_dims))
    model.add(BatchNormalization())
    model.add(Activation(act_func))
    model.add(Dropout(0.3))
    
    model.add(Dense(input_dims//2))
    model.add(BatchNormalization())
    model.add(Activation(act_func))
    model.add(Dropout(0.3))
    
    model.add(Dense(input_dims//4, activation=act_func))
    
    # output layer (y_pred)
    model.add(Dense(1, activation='linear'))
    
    # compile this model
    model.compile(loss='mean_squared_error', # one may use 'mean_absolute_error' as alternative
                  optimizer='adam',
                  metrics=[r2_keras] # you can add several if needed
                 )
    
    # Visualize NN architecture
    print(model.summary())
    return model

In [32]:
estimator = KerasRegressor(
    build_fn=model, 
    nb_epoch=100, 
    batch_size=20,
    verbose=1
)

** Gradient Boosting Regressor **

In [33]:
from sklearn.ensemble import GradientBoostingRegressor

In [34]:
train_X, eval_X, train_y, eval_y = train_test_split(train_feats_encode, train_labels)

In [35]:
def check(model):
    pred_train = model.predict(train_X)
    score_train = r2_score(train_y, pred_train)
    pred_eval = model.predict(eval_X)
    score_eval = r2_score(eval_y, pred_eval)
    print(model.__class__.__name__)
    print("r2 train = ", score_train)
    print("r2 eval = ", score_eval)
    print("====================================================\n")
    return score_train, score_eval, pred_train, pred_eval

In [36]:
def eval_model():
    
    model_theilsen = TheilSenRegressor(random_state=42)
    model_theilsen.fit(train_X, train_y)
    train_theilsen, test_theilsen, pred_train_theilsen, pred_eval_theilsen = check(model_theilsen)

    model_ransac = RANSACRegressor(random_state=42)
    model_ransac.fit(train_X, train_y)
    train_ransac, test_ransac, pred_train_ransac, pred_eval_ransac = check(model_ransac)

    model_huber = HuberRegressor()
    model_huber.fit(train_X, train_y)
    train_huber, test_huber, pred_train_huber, pred_eval_huber = check(model_huber)

    
    model_linear = LinearRegression()
    model_linear.fit(train_X, train_y)
    train_lr, test_lr, pred_train_lr, pred_eval_lr = check(model_linear)

    model_sgd = SGDRegressor()
    model_sgd.fit(train_X, train_y)
    train_sgd, test_sgd, pred_train_sgd, pred_eval_sgd = check(model_sgd)
        
    model_xgb = XGBRegressor(seed = 0,
      colsample_bytree = 0.7,
      subsample = 0.9,
      eta = 0.005,
      max_depth = 4,
      num_parallel_tree = 1,
      min_child_weight = 1, objective='reg:linear', base_score=np.mean(train_labels))
    model_xgb.fit(train_X, train_y)
    train_xgb, test_xgb, pred_train_xgb, pred_eval_xgb = check(model_xgb)
    
    model_gbr = GradientBoostingRegressor()
    model_gbr.fit(train_X, train_y)
    train_gbr, test_gbr, pred_train_gbr, pred_eval_gbr = check(model_gbr)
    
    estimator.fit(
        train_feats_encode, 
        train_labels, 
        epochs=300, # increase it to 20-100 to get better results
        verbose=2,
        shuffle=True
    )

    train_dl, test_dl, pred_train_dl, pred_eval_dl = check(estimator)
    
    columns=['lr', 'sgd', 'xgb', 'gbr', 'dl', 'thelisen', 'ransac', 'huber']
   
    test_res = (pred_eval_lr, pred_eval_sgd, pred_eval_xgb, pred_eval_gbr, pred_eval_dl, pred_eval_theilsen, pred_eval_ransac, pred_eval_huber)
    
    train_res = (pred_train_lr, pred_train_sgd, pred_train_xgb, pred_train_gbr, pred_train_dl, pred_train_theilsen, pred_train_ransac, pred_train_huber)
    
    train_res_pd = pd.DataFrame(data=np.column_stack(train_res),
                  columns=columns)
    
    test_res_pd = pd.DataFrame(data=np.column_stack(test_res),
                  columns=columns)
    models = [model_linear, model_sgd, model_xgb, model_gbr, estimator, model_theilsen, model_ransac, model_huber]
    return train_res_pd, test_res_pd, models


In [37]:
train_pd, eval_pd, models = eval_model()

TheilSenRegressor
r2 train =  0.159201332928
r2 eval =  0.293265801864

RANSACRegressor
r2 train =  0.427183244977
r2 eval =  0.449259284886

HuberRegressor
r2 train =  0.158524138962
r2 eval =  0.298549780541

LinearRegression
r2 train =  0.546072065004
r2 eval =  0.591935097314

SGDRegressor
r2 train =  -3.71882219809e+30
r2 eval =  -3.70582255637e+30

XGBRegressor
r2 train =  0.656287917263
r2 eval =  0.599995096358

GradientBoostingRegressor
r2 train =  0.621485051292
r2 eval =  0.604012222866

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 14)                210       
_________________________________________________________________
batch_normalization_4 (Batch (None, 14)                56        
_________________________________________________________________
activation_4 (Activation)    (None, 14)                0         
__________________________________

0s - loss: 820.3853 - r2_keras: -5.1381e+00
Epoch 91/300
0s - loss: 783.2649 - r2_keras: -4.9187e+00
Epoch 92/300
0s - loss: 747.4603 - r2_keras: -4.5930e+00
Epoch 93/300
0s - loss: 712.7452 - r2_keras: -4.2706e+00
Epoch 94/300
0s - loss: 679.1980 - r2_keras: -4.0276e+00
Epoch 95/300
0s - loss: 646.9456 - r2_keras: -3.7933e+00
Epoch 96/300
0s - loss: 615.8077 - r2_keras: -3.5029e+00
Epoch 97/300
0s - loss: 585.8889 - r2_keras: -3.3047e+00
Epoch 98/300
0s - loss: 557.1025 - r2_keras: -3.0205e+00
Epoch 99/300
0s - loss: 529.4222 - r2_keras: -2.8276e+00
Epoch 100/300
0s - loss: 502.9619 - r2_keras: -2.7052e+00
Epoch 101/300
0s - loss: 477.6463 - r2_keras: -2.4282e+00
Epoch 102/300
0s - loss: 453.4839 - r2_keras: -2.2844e+00
Epoch 103/300
0s - loss: 430.4078 - r2_keras: -2.1015e+00
Epoch 104/300
0s - loss: 408.4279 - r2_keras: -1.9568e+00
Epoch 105/300
0s - loss: 387.5805 - r2_keras: -1.7519e+00
Epoch 106/300
0s - loss: 367.8289 - r2_keras: -1.6269e+00
Epoch 107/300
0s - loss: 349.0979 - r

0s - loss: 160.7389 - r2_keras: -7.3174e-02
Epoch 233/300
0s - loss: 160.7323 - r2_keras: -5.4846e-02
Epoch 234/300
0s - loss: 160.7365 - r2_keras: -6.5165e-02
Epoch 235/300
0s - loss: 160.7371 - r2_keras: -7.0182e-02
Epoch 236/300
0s - loss: 160.7396 - r2_keras: -6.0438e-02
Epoch 237/300
0s - loss: 160.7342 - r2_keras: -5.9869e-02
Epoch 238/300
0s - loss: 160.7442 - r2_keras: -6.1231e-02
Epoch 239/300
0s - loss: 160.7362 - r2_keras: -5.0825e-02
Epoch 240/300
0s - loss: 160.7613 - r2_keras: -6.2342e-02
Epoch 241/300
0s - loss: 160.7499 - r2_keras: -6.7165e-02
Epoch 242/300
0s - loss: 160.7245 - r2_keras: -5.3012e-02
Epoch 243/300
0s - loss: 160.7446 - r2_keras: -6.3497e-02
Epoch 244/300
0s - loss: 160.7511 - r2_keras: -5.9595e-02
Epoch 245/300
0s - loss: 160.7371 - r2_keras: -5.2636e-02
Epoch 246/300
0s - loss: 160.7342 - r2_keras: -6.0090e-02
Epoch 247/300
0s - loss: 160.7238 - r2_keras: -5.4447e-02
Epoch 248/300
0s - loss: 160.7570 - r2_keras: -6.7809e-02
Epoch 249/300
0s - loss: 160

In [38]:
print(r2_score(train_y, train_pd.median(axis=1)))
print(r2_score(eval_y, eval_pd.median(axis=1)))

0.562194842033
0.586563854392


In [39]:
print(r2_score(train_y, train_pd[['lr', 'sgd', 'xgb', 'gbr', 'dl', 'thelisen', 'ransac']].median(axis=1)))
print(r2_score(eval_y, eval_pd[['lr', 'sgd', 'xgb', 'gbr', 'dl', 'thelisen', 'ransac']].median(axis=1)))

0.582874232858
0.596360139532


In [40]:
models

[LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False),
 SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
        fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
        loss='squared_loss', n_iter=5, penalty='l2', power_t=0.25,
        random_state=None, shuffle=True, verbose=0, warm_start=False),
 XGBRegressor(base_score=100.66931812782121, booster='gbtree',
        colsample_bylevel=1, colsample_bytree=0.7, eta=0.005, gamma=0,
        learning_rate=0.1, max_delta_step=0, max_depth=4,
        min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
        nthread=1, num_parallel_tree=1, objective='reg:linear',
        random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
        seed=0, silent=True, subsample=0.9),
 GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
              max_leaf_nodes=None, min_impurity_spli

In [41]:
preds = []
for model in models:
    preds.append(model.predict(test_feats_encode))
    
dt = pd.DataFrame(data=np.array(preds).T)

4040/4209 [===========================>..] - ETA: 0s

In [44]:
res = dt.median(axis=1)

In [45]:
output = pd.DataFrame({'id': test['ID'].astype(np.int32)})


In [47]:
output['y'] = res

In [48]:
output.to_csv('results/xgboost/ensemble[%s][%s].csv' % ('ensemble','qweqweqwe123'), index=False)